In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
project_id = 'bigquerycourse-464720'

In [ ]:
%load_ext google.colab.data_table

#Active Customers Calculation


## 🔍 Exploring and Scoping the Dataset

>Let's explore Users and Orders table

In [ ]:
%%bigquery --project {project_id}
SELECT
  *
FROM
  `bigquery-public-data.thelook_ecommerce.users`
LIMIT
  5
# This is a table of All Unique Users with properities like id and others.
# But, we need aditional data because theirs no columns related to orders for each user.

In [ ]:
%%bigquery --project {project_id}
SELECT
  *
from
  `bigquery-public-data.thelook_ecommerce.orders`
LIMIT
  5
# This is a table of All UNIQUE orders that Users with other details like order_id and user_id.
# The good is that we have the user_id. With this we can relate the USERs table with the Order table.

> Before moving foward, the KPI to be calculated need to be in a certain period (quarter). Now let's know the time period of the Order Table and select one Quarter.

In [ ]:
#We realize that is a created_at column. To move foward
#we need to know the time period.

%%bigquery --project {project_id}
SELECT
  min(date(created_at)) as First_Date,
  max(date(created_at)) as Last_Date,
  max(date(created_at))-min(date(created_at)) as Time_Period,
  DATE_DIFF(max(date(created_at)), min(date(created_at)), DAY) as Time_Period_Days,
from
  `bigquery-public-data.thelook_ecommerce.orders`

#The records go from 2019-01-13 to 2025-09-02 and includes 2424 days of transactions.
#note that using DATE_DIFF will return the bays between in a integer data type.

> Now that we have confirmed the date range of the data, lets select the first quarter of 2025. We will run just to checking pourposuses.

In [ ]:
#Lets check the info
%%bigquery --project {project_id}
SELECT
  min(date(created_at)) as Order_Date,
  max(date(created_at)) as Order_Date,
  DATE_DIFF(max(date(created_at)), min(date(created_at)), DAY) as Time_Period_Days,
from
  `bigquery-public-data.thelook_ecommerce.orders`
where
  date(created_at) between '2025-01-01' and '2025-03-31'
#

## 💻 Creating the Query

>Now that we have confirmed the date range of the data , we can move on to creating a query that identifies active users within the first quarter of 2025.

In [ ]:
%%bigquery --project {project_id}
SELECT
  distinct (U.id) AS User_ID,
  COUNT(O.order_id) ORDERS
FROM
  bigquery-public-data.thelook_ecommerce.users U
LEFT JOIN
  bigquery-public-data.thelook_ecommerce.orders O
ON
  U.id = O.user_id
WHERE
  O.order_id IS NOT NULL and date(O.created_at) between '2025-01-01' and '2025-03-31'
GROUP BY
  User_ID
ORDER BY
  User_ID asc

In [ ]:
%%bigquery --project {project_id}
SELECT
    COUNT(User_ID) AS Total_Users
FROM (
    SELECT
  distinct (U.id) AS User_ID,
  COUNT(O.order_id) ORDERS
FROM
  bigquery-public-data.thelook_ecommerce.users U
LEFT JOIN
  bigquery-public-data.thelook_ecommerce.orders O
ON
  U.id = O.user_id
WHERE
  O.order_id IS NOT NULL
  AND date(O.created_at) between '2025-01-01' and '2025-03-31'
GROUP BY
  User_ID
ORDER BY
  User_ID asc
) AS subquery_result